In [2]:
import gc
import pandas as pd
import numpy as np
import os

from glob import glob
from tqdm import tqdm_notebook, tqdm

pd.set_option('display.max_columns', 100)

from utils import loadpkl, to_feature, line_notify

%matplotlib inline

In [2]:
files = sorted(glob('../feats/*.feather'))
df = pd.concat([pd.read_feather(f) for f in tqdm(files, mininterval=60)], axis=1)

  0%|          | 0/24 [00:00<?, ?it/s]/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pandas/io/feather_format.py:124: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  nthreads=int_use_threads)
/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/pyarrow/pandas_compat.py:751: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
100%|██████████| 24/24 [00:06<00:00,  3.98it/s]


In [3]:
df

,cat_id,d,date,demand,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,id,index,is_test,item_id,month,seasonality,sell_price,snap_CA,snap_TX,snap_WI,state_id,store_id,wday,wm_yr_wk,year
0,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_001_CA_1_validation,31410874,0,HOBBIES_1_001,1,-0.999853,8.260000,1,1,0,CA,CA_1,6,11448,2015
1,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_002_CA_1_validation,31410875,0,HOBBIES_1_002,1,-0.999853,3.970000,1,1,0,CA,CA_1,6,11448,2015
2,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_003_CA_1_validation,31410876,0,HOBBIES_1_003,1,-0.999853,2.970000,1,1,0,CA,CA_1,6,11448,2015
3,HOBBIES,d_1434,2015-01-01,2,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_004_CA_1_validation,31410877,0,HOBBIES_1_004,1,-0.999853,4.640000,1,1,0,CA,CA_1,6,11448,2015
4,HOBBIES,d_1434,2015-01-01,3,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_005_CA_1_validation,31410878,0,HOBBIES_1_005,1,-0.999853,2.880000,1,1,0,CA,CA_1,6,11448,2015
5,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_006_CA_1_validation,31410879,0,HOBBIES_1_006,1,-0.999853,1.000000,1,1,0,CA,CA_1,6,11448,2015
6,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_007_CA_1_validation,31410880,0,HOBBIES_1_007,1,-0.999853,7.880000,1,1,0,CA,CA_1,6,11448,2015
7,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_008_CA_1_validation,31410881,0,HOBBIES_1_008,1,-0.999853,0.460000,1,1,0,CA,CA_1,6,11448,2015
8,HOBBIES,d_1434,2015-01-01,2,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_009_CA_1_validation,31410882,0,HOBBIES_1_009,1,-0.999853,1.770000,1,1,0,CA,CA_1,6,11448,2015
9,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_010_CA_1_validation,31410883,0,HOBBIES_1_010,1,-0.999853,2.970000,1,1,0,CA,CA_1,6,11448,2015


In [9]:
df[df.is_test==1]['date'].min()

Timestamp('2016-04-25 00:00:00')

In [11]:
df[df.date>='2016-04-25']['is_test'].value_counts()

1    1707440
Name: is_test, dtype: int64

In [12]:
df[df.is_test==1]

,cat_id,d,date,demand,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,id,index,is_test,item_id,month,seasonality,sell_price,snap_CA,snap_TX,snap_WI,state_id,store_id,wday,wm_yr_wk,year
14531626,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_001_CA_1_validation,45942500,1,HOBBIES_1_001,4,0.408304,8.380000,0,0,0,CA,CA_1,3,11613,2016
14531627,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_002_CA_1_validation,45942501,1,HOBBIES_1_002,4,0.408304,3.970000,0,0,0,CA,CA_1,3,11613,2016
14531628,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_003_CA_1_validation,45942502,1,HOBBIES_1_003,4,0.408304,2.970000,0,0,0,CA,CA_1,3,11613,2016
14531629,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_004_CA_1_validation,45942503,1,HOBBIES_1_004,4,0.408304,4.640000,0,0,0,CA,CA_1,3,11613,2016
14531630,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_005_CA_1_validation,45942504,1,HOBBIES_1_005,4,0.408304,2.880000,0,0,0,CA,CA_1,3,11613,2016
14531631,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_006_CA_1_validation,45942505,1,HOBBIES_1_006,4,0.408304,0.960000,0,0,0,CA,CA_1,3,11613,2016
14531632,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_007_CA_1_validation,45942506,1,HOBBIES_1_007,4,0.408304,7.880000,0,0,0,CA,CA_1,3,11613,2016
14531633,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_008_CA_1_validation,45942507,1,HOBBIES_1_008,4,0.408304,0.480000,0,0,0,CA,CA_1,3,11613,2016
14531634,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_009_CA_1_validation,45942508,1,HOBBIES_1_009,4,0.408304,1.770000,0,0,0,CA,CA_1,3,11613,2016
14531635,HOBBIES,d_1914,2016-04-25,0,HOBBIES_1,NaN,NaN,NaN,NaN,HOBBIES_1_010_CA_1_validation,45942509,1,HOBBIES_1_010,4,0.408304,2.970000,0,0,0,CA,CA_1,3,11613,2016


In [13]:
df.sort_values('date')

,cat_id,d,date,demand,dept_id,event_name_1,event_name_2,event_type_1,event_type_2,id,index,is_test,item_id,month,seasonality,sell_price,snap_CA,snap_TX,snap_WI,state_id,store_id,wday,wm_yr_wk,year
0,HOBBIES,d_1434,2015-01-01,0,HOBBIES_1,26.0,NaN,2.0,NaN,HOBBIES_1_001_CA_1_validation,31410874,0,HOBBIES_1_001,1,-0.999853,8.260000,1,1,0,CA,CA_1,6,11448,2015
19711,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_311_TX_3_validation,31430585,0,FOODS_2_311,1,-0.999853,2.420000,1,1,0,TX,TX_3,6,11448,2015
19710,FOODS,d_1434,2015-01-01,1,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_309_TX_3_validation,31430584,0,FOODS_2_309,1,-0.999853,2.440000,1,1,0,TX,TX_3,6,11448,2015
19709,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_308_TX_3_validation,31430583,0,FOODS_2_308,1,-0.999853,2.220000,1,1,0,TX,TX_3,6,11448,2015
19708,FOODS,d_1434,2015-01-01,1,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_307_TX_3_validation,31430582,0,FOODS_2_307,1,-0.999853,2.500000,1,1,0,TX,TX_3,6,11448,2015
19707,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_306_TX_3_validation,31430581,0,FOODS_2_306,1,-0.999853,4.140000,1,1,0,TX,TX_3,6,11448,2015
19706,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_305_TX_3_validation,31430580,0,FOODS_2_305,1,-0.999853,8.960000,1,1,0,TX,TX_3,6,11448,2015
19705,FOODS,d_1434,2015-01-01,1,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_304_TX_3_validation,31430579,0,FOODS_2_304,1,-0.999853,3.970000,1,1,0,TX,TX_3,6,11448,2015
19704,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_303_TX_3_validation,31430578,0,FOODS_2_303,1,-0.999853,2.180000,1,1,0,TX,TX_3,6,11448,2015
19703,FOODS,d_1434,2015-01-01,0,FOODS_2,26.0,NaN,2.0,NaN,FOODS_2_302_TX_3_validation,31430577,0,FOODS_2_302,1,-0.999853,9.980000,1,1,0,TX,TX_3,6,11448,2015


In [14]:
# split train & test
train_df = df[df['date']<'2016-04-25']
test_df = df[df['date']>='2016-04-25']
del df
gc.collect()

0

In [20]:
from utils import NUM_FOLDS, FEATS_EXCLUDED, COLS_TEST1, COLS_TEST2

In [18]:
pred = test_df[['id','d','demand']].pivot(index='id', columns='d', values='demand').reset_index()

In [23]:
preds1 = pred[['id']+COLS_TEST1]
preds2 = pred[['id']+COLS_TEST2]

In [26]:
preds1.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]
preds2.columns = ['id'] + ['F' + str(d + 1) for d in range(28)]

In [29]:
preds2['id']= preds2['id'].str.replace('_validation','_evaluation')

/home/fujiwara/.pyenv/versions/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [30]:
preds1.append(preds2)

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,FOODS_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,FOODS_1_001_CA_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,FOODS_1_001_CA_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,FOODS_1_001_CA_4_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,FOODS_1_001_TX_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,FOODS_1_001_TX_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,FOODS_1_001_TX_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,FOODS_1_001_WI_1_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,FOODS_1_001_WI_2_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,FOODS_1_001_WI_3_validation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
